## DSC540 project milestone 3
#### Author: Xin Tang
### date: 2/2/2024

#### Perform at least 5 data transformation and/or cleansing steps to your website data

###### the website has the information of all Texas airport and related info
I will clean up data, find the correct airport for city of Austin

In [1]:
# import related package
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

###### Step1. extract table from website and exam which table is what I need

In [3]:
list_of_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_airports_in_Texas", header=0)
for t in list_of_df: print(t.shape)

(457, 7)
(0, 2)
(4, 2)
(7, 2)


In [4]:
# cross check with the table on the webpage, looks like the first table is what I needed.

###### Step2. get the correct table and adjust the header

In [4]:
#extra the correct table, remove space out of essential variable names to avoid calling variable name with space
df_new = list_of_df[0]
df_new.rename(columns = {'City served':'City'}, inplace = True)
df_new.rename(columns = {'Airport name':'Airport_name'}, inplace = True)
print(df_new.head())
df_new.shape

       City  FAA IATA  ICAO  \
0       NaN  NaN  NaN   NaN   
1   Abilene  ABI  ABI  KABI   
2  Amarillo  AMA  AMA  KAMA   
3    Austin  AUS  AUS  KAUS   
4  Beaumont  BPT  BPT  KBPT   

                                        Airport_name Role  Enplanements (2021)  
0              Commercial service – primary airports  NaN                  NaN  
1                           Abilene Regional Airport  P-N              75402.0  
2        Rick Husband Amarillo International Airport  P-N             303904.0  
3             Austin–Bergstrom International Airport  P-L            6666215.0  
4  Jack Brooks Regional Airport (was Southeast Te...  P-N              21914.0  


(457, 7)

###### Step3. drop first few records since they are not linked to any specific airports

In [5]:
#df_new.drop([0], axis=0, inplace=True)
df_new.drop(df_new.index[0], inplace=True)
df_new.shape

(456, 7)

###### Step4. exam the dataframe for duplicate and NAs. Fix casing if needed.

In [6]:
#Check datatable health, check airport code are upper case
cols = df_new.columns
def Unique_Values():
    for i in np.arange(0,len(cols)):
        print('There are {} of unique values in {} column out of {}'.format(df_new[cols[i]].nunique(), cols[i], len(df_new)))
print(Unique_Values())
#df_new.info()
print('variables with NA values\n', df_new.isna().sum())
print('\n it is {a} that all airport codes are upper case'.format(a = df_new["IATA"].str.isupper().all()))

There are 359 of unique values in City column out of 456
There are 447 of unique values in FAA column out of 456
There are 126 of unique values in IATA column out of 456
There are 187 of unique values in ICAO column out of 456
There are 456 of unique values in Airport_name column out of 456
There are 7 of unique values in Role column out of 456
There are 79 of unique values in Enplanements (2021) column out of 456
None
variables with NA values
 City                     7
FAA                      8
IATA                   329
ICAO                   268
Airport_name             0
Role                   245
Enplanements (2021)    247
dtype: int64

 it is True that all airport codes are upper case


###### summary:
1. there are some records without city info filled, but we can filter them out
2. There are many cities with duplicate. That's could due to mulltiple airports in the same city. since airport name has no duplicate.
3. airport names are not empty, so no need to do fuzzy matching.
4. for this data, no need to check outliners.

###### Step5. Find airport serving Austin

In [7]:
# find airport serving Austin, there are 3 airports in Austin.
Austin = df_new[df_new['City'] =='Austin']
print(Austin)

       City  FAA IATA  ICAO  \
3    Austin  AUS  AUS  KAUS   
226  Austin  EDC  NaN  KEDC   
424  Austin  AUS  AUS  KAUS   

                                          Airport_name Role  \
3               Austin–Bergstrom International Airport  P-L   
226  Austin Executive Airport (was Bird's Nest Airp...  NaN   
424  Robert Mueller Municipal Airport (closed 1999)...  NaN   

     Enplanements (2021)  
3              6666215.0  
226                  NaN  
424                  NaN  


###### Step6. remove unrelevant columns and keep essential ones only.

In [8]:
Austin_df = Austin[['City','FAA', 'IATA','Airport_name','Role']]
Austin_df

,City,FAA,IATA,Airport_name,Role
3,Austin,AUS,AUS,Austin–Bergstrom International Airport,P-L
226,Austin,EDC,NaN,Austin Executive Airport (was Bird's Nest Airp...,NaN
424,Austin,AUS,AUS,Robert Mueller Municipal Airport (closed 1999)...,NaN


###### Step 7. only P and L are large commerical airport, filter out non-commericial airport, left is what I am interested

In [45]:
AUS = Austin_df[Austin_df['Role']== 'P-L']
#A = AUS['IATA']
print("Austin airport code is:", AUS['IATA'].loc[AUS.index[0]] )

Austin airport code is: AUS


### End of milestone3

In [9]:
Austin_df.to_csv('IATA_code.csv')